
# Project: Investigate a Dataset (TMDb Movie Data)

###### By Karim El-Dweky

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

The purpose of this project is to put in practice what I learned in data wrangling data section from Udacity Data Analysis Nanodegree program. The dataset that is wrangled contains information about 10,000 movies collected from The Movie Database (TMDb), including user ratings and revenue.

In [247]:
# importing required libraries 
import pandas as pd
import numpy as np
import requests
import re
import matplotlib.pyplot as plt
import datetime
import os
import seaborn as sns
from scipy import stats
from functools import reduce
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling


### 1. Data Gathering
Kaggle have removed the original version of this dataset per a DMCA takedown request from IMDB. In order to minimize the impact, They're replacing it with a similar set of films and data fields from The Movie Database (TMDb) in accordance with their terms of use.
- **TMDb Movie Data**
    - This file (tmdb-movies.csv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd1c4c_tmdb-movies/tmdb-movies.csv.
    - The file is cleaned from original data on https://www.kaggle.com/tmdb/tmdb-movie-metadata.


In [248]:
# URL downloaded programatically 
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/October/59dd1c4c_tmdb-movies/tmdb-movies.csv"
file_name = os.path.basename(url)
response = requests.get(url)

if not os.path.isfile(file_name):
    with open(file_name, 'wb') as f:
        f.write(response.content)

# Read CSV file
tmdb_df = pd.read_csv(file_name)
tmdb_df.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


### 2. Data Assessment 
Data Assessment consists of two main steps (Visual - Programmatic) which help in exploring the gathered data and finding the anamolies points that needs to be cleaned. 

#### a. Visual Assessment

In [249]:
tmdb_df

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,21,tt0060371,0.080598,0,0,The Endless Summer,Michael Hynson|Robert August|Lord 'Tally Ho' B...,NaN,Bruce Brown,NaN,...,"The Endless Summer, by Bruce Brown, is one of ...",95,Documentary,Bruce Brown Films,6/15/66,11,7.4,1966,0.000000e+00,0.000000e+00
10862,20379,tt0060472,0.065543,0,0,Grand Prix,James Garner|Eva Marie Saint|Yves Montand|Tosh...,NaN,John Frankenheimer,Cinerama sweeps YOU into a drama of speed and ...,...,Grand Prix driver Pete Aron is fired by his te...,176,Action|Adventure|Drama,Cherokee Productions|Joel Productions|Douglas ...,12/21/66,20,5.7,1966,0.000000e+00,0.000000e+00
10863,39768,tt0060161,0.065141,0,0,Beregis Avtomobilya,Innokentiy Smoktunovskiy|Oleg Efremov|Georgi Z...,NaN,Eldar Ryazanov,NaN,...,An insurance agent who moonlights as a carthie...,94,Mystery|Comedy,Mosfilm,1/1/66,11,6.5,1966,0.000000e+00,0.000000e+00
10864,21449,tt0061177,0.064317,0,0,"What's Up, Tiger Lily?",Tatsuya Mihashi|Akiko Wakabayashi|Mie Hama|Joh...,NaN,Woody Allen,WOODY ALLEN STRIKES BACK!,...,"In comic Woody Allen's film debut, he took the...",80,Action|Comedy,Benedict Pictures Corp.,11/2/66,22,5.4,1966,0.000000e+00,0.000000e+00


#### b. Programmatic Assessment

In [250]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

In [251]:
tmdb_df.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


In [252]:
sum(tmdb_df['id'].duplicated())

1

In [253]:
tmdb_df[tmdb_df['id'].duplicated()]

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
2090,42194,tt0411951,0.59643,30000000,967000,TEKKEN,Jon Foo|Kelly Overton|Cary-Hiroyuki Tagawa|Ian...,NaN,Dwight H. Little,Survival is no game,...,"In the year of 2039, after World Wars destroy ...",92,Crime|Drama|Action|Thriller|Science Fiction,Namco|Light Song Films,3/20/10,110,5.0,2010,30000000.0,967000.0


In [254]:
tmdb_df.popularity.value_counts()

0.109305    2
0.114027    2
0.126182    2
0.247926    2
0.410235    2
           ..
0.645437    1
0.088796    1
0.155075    1
0.596755    1
0.234375    1
Name: popularity, Length: 10814, dtype: int64

In [255]:
len(tmdb_df[tmdb_df['popularity'] <= 1])

9110

In [256]:
len(tmdb_df[tmdb_df['popularity'] > 1])

1756

In [257]:
tmdb_df.popularity.sum()

7024.227383999999

In [258]:
tmdb_df.vote_average.value_counts()

6.1    496
6.0    495
5.8    486
5.9    473
6.2    464
      ... 
8.9      1
8.6      1
9.2      1
8.7      1
2.0      1
Name: vote_average, Length: 72, dtype: int64

In [259]:
len(tmdb_df[tmdb_df['vote_average'] <= 5.97])

5056

In [260]:
len(tmdb_df[tmdb_df['vote_average'] > 5.97])

5810

In [261]:
len(tmdb_df[tmdb_df['vote_average'] == 0])

0

In [262]:
tmdb_df.sample(5)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
1696,14164,tt1098327,0.322320,100000000,0,Dragonball Evolution,Justin Chatwin|Joon Park|Jamie Chung|Emmy Ross...,NaN,James Wong,The legend comes to life.,...,The young warrior Son Goku sets out on a quest...,85,Action|Adventure|Fantasy|Science Fiction|Thriller,Ingenious Film Partners|Twentieth Century Fox ...,4/1/09,261,3.1,2009,1.016400e+08,0.000000e+00
6302,14517,tt0366780,0.663398,4000000,866999,MirrorMask,Stephanie Leonidas|Jason Barry|Rob Brydon|Gina...,NaN,Dave McKean,The Other Side of Reason. The Other Side of Fa...,...,"In a fantasy world of opposing kingdoms, a 15-...",101,Adventure|Fantasy|Drama|Science Fiction|Family,Destination Films,1/25/05,44,6.5,2005,4.466253e+06,9.680593e+05
1010,241843,tt2490326,0.689942,0,0,Cooties,Elijah Wood|Alison Pill|Rainn Wilson|Jack McBr...,NaN,Cary Murnion|Jonathan Milott,Please don't feed the children.,...,A mysterious virus hits an isolated elementary...,88,Action|Comedy|Horror,SpectreVision|Glacier Films,10/10/14,136,5.2,2014,0.000000e+00,0.000000e+00
10338,8291,tt0107840,0.313792,14000000,27515786,Poetic Justice,Janet Jackson|Tupac Shakur|Regina King|Joe Tor...,http://www.janetjackson.com,John Singleton,A Street Romance.,...,"In this film, we see the world through the eye...",109,Drama|Romance,Columbia Pictures,7/23/93,24,6.8,1993,2.113258e+07,4.153425e+07
4195,4476,tt0110322,1.253496,30000000,160638883,Legends of the Fall,Brad Pitt|Anthony Hopkins|Aidan Quinn|Julia Or...,NaN,Edward Zwick,After the Fall from Innocence the Legend begins.,...,An epic tale of three brothers and their fathe...,133,Drama|Western|War|Romance,Bedford Falls Productions|TriStar Pictures|Pan...,12/16/94,380,7.1,1994,4.413335e+07,2.363177e+08


In [263]:
tmdb_df.isna().sum()

id                         0
imdb_id                   10
popularity                 0
budget                     0
revenue                    0
original_title             0
cast                      76
homepage                7930
director                  44
tagline                 2824
keywords                1493
overview                   4
runtime                    0
genres                    23
production_companies    1030
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj                 0
revenue_adj                0
dtype: int64

In [264]:
len(tmdb_df[tmdb_df['budget_adj'] == 0])

5696

In [265]:
len(tmdb_df[tmdb_df['revenue_adj'] == 0])

6016

In [266]:
len(tmdb_df[tmdb_df['budget'] == 0])

5696

In [267]:
len(tmdb_df[tmdb_df['revenue'] == 0])

6016

In [268]:
tmdb_df.isin([0]).sum()

id                         0
imdb_id                    0
popularity                 0
budget                  5696
revenue                 6016
original_title             0
cast                       0
homepage                   0
director                   0
tagline                    0
keywords                   0
overview                   0
runtime                   31
genres                     0
production_companies       0
release_date               0
vote_count                 0
vote_average               0
release_year               0
budget_adj              5696
revenue_adj             6016
dtype: int64

#### c. Data Assessment Report

##### - Quality Issues:

   1. Dropping Dublicated **id**.
   2. Dropping unneeded columns.
   3. Converting zeros in **budget_adj** & **revenue_adj** to np-NaN.
   4. Dropping all rows with Nan Values.
   5. Converting **release_date** from object into datetime.

##### - Tideness Issues:

   6. Separate **release_date** into **release_day**, **release_month** and **release_year** (3 columns) and Dropping **release_date**.

### 3. Data Cleaning

#### a. Taking a copy to be cleaned:

In [285]:
tmdb_df_cleaned = tmdb_df.copy()

#### b. Cleaning Quality Issues:

###### Define

1. Dropping Dublicated **id**.

###### Code

In [286]:
# sorting by id 
tmdb_df_cleaned.sort_values('id', inplace = True) 
  
# dropping ALL duplicates values 
tmdb_df_cleaned.drop_duplicates(subset ='id', 
                     keep = False, inplace = True) 

###### Test

In [287]:
sum(tmdb_df_cleaned['id'].duplicated())

0

###### Define

2. Dropping unneeded columns.

###### Code

In [288]:
# Dropping (imdb_id, budget, revenue, homepage, tagline, keywords, overview, release_year) coloumns 

unneeded_coulmns = ['imdb_id',
                    'budget',
                    'revenue',
                    'homepage',
                    'tagline',
                    'keywords',
                    'overview',
                    'release_year']

tmdb_df_cleaned.drop(unneeded_coulmns, axis=1, inplace=True)

###### Test

In [289]:
tmdb_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10864 entries, 8088 to 3460
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10864 non-null  int64  
 1   popularity            10864 non-null  float64
 2   original_title        10864 non-null  object 
 3   cast                  10788 non-null  object 
 4   director              10820 non-null  object 
 5   runtime               10864 non-null  int64  
 6   genres                10841 non-null  object 
 7   production_companies  9834 non-null   object 
 8   release_date          10864 non-null  object 
 9   vote_count            10864 non-null  int64  
 10  vote_average          10864 non-null  float64
 11  budget_adj            10864 non-null  float64
 12  revenue_adj           10864 non-null  float64
dtypes: float64(4), int64(3), object(6)
memory usage: 1.2+ MB


###### Define

3. Converting zeros in **budget_adj** & **revenue_adj** to np-NaN.

###### Code

In [290]:
# Converting zeros in budget_adj to np-NaN
tmdb_df_cleaned['budget_adj'].replace(0, np.nan, inplace = True)
  
# Converting zeros in revenue_adj to np-NaN
tmdb_df_cleaned['revenue_adj'].replace(0, np.nan, inplace = True)

###### Test

In [291]:
tmdb_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10864 entries, 8088 to 3460
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10864 non-null  int64  
 1   popularity            10864 non-null  float64
 2   original_title        10864 non-null  object 
 3   cast                  10788 non-null  object 
 4   director              10820 non-null  object 
 5   runtime               10864 non-null  int64  
 6   genres                10841 non-null  object 
 7   production_companies  9834 non-null   object 
 8   release_date          10864 non-null  object 
 9   vote_count            10864 non-null  int64  
 10  vote_average          10864 non-null  float64
 11  budget_adj            5168 non-null   float64
 12  revenue_adj           4848 non-null   float64
dtypes: float64(4), int64(3), object(6)
memory usage: 1.2+ MB


###### Define

4. Dropping all rows with Nan Values.

###### Code

In [292]:
# Dropping all rows with Nan Values.
tmdb_df_cleaned.dropna(subset=[n for n in tmdb_df_cleaned if n != 'revenue_adj'], inplace = True)

###### Test

In [293]:
tmdb_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5020 entries, 8088 to 3460
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5020 non-null   int64  
 1   popularity            5020 non-null   float64
 2   original_title        5020 non-null   object 
 3   cast                  5020 non-null   object 
 4   director              5020 non-null   object 
 5   runtime               5020 non-null   int64  
 6   genres                5020 non-null   object 
 7   production_companies  5020 non-null   object 
 8   release_date          5020 non-null   object 
 9   vote_count            5020 non-null   int64  
 10  vote_average          5020 non-null   float64
 11  budget_adj            5020 non-null   float64
 12  revenue_adj           3804 non-null   float64
dtypes: float64(4), int64(3), object(6)
memory usage: 549.1+ KB


###### Define

5. Converting **release_date** coloumn from object into datetime.

###### Code

In [294]:
# Converting from object into datetime

tmdb_df_cleaned['release_date'] =  pd.to_datetime(tmdb_df_cleaned['release_date'], infer_datetime_format=True)

###### Test

In [295]:
tmdb_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5020 entries, 8088 to 3460
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    5020 non-null   int64         
 1   popularity            5020 non-null   float64       
 2   original_title        5020 non-null   object        
 3   cast                  5020 non-null   object        
 4   director              5020 non-null   object        
 5   runtime               5020 non-null   int64         
 6   genres                5020 non-null   object        
 7   production_companies  5020 non-null   object        
 8   release_date          5020 non-null   datetime64[ns]
 9   vote_count            5020 non-null   int64         
 10  vote_average          5020 non-null   float64       
 11  budget_adj            5020 non-null   float64       
 12  revenue_adj           3804 non-null   float64       
dtypes: datetime64[n

In [296]:
tmdb_df_cleaned.head()

,id,popularity,original_title,cast,director,runtime,genres,production_companies,release_date,vote_count,vote_average,budget_adj,revenue_adj
8088,5,1.234890,Four Rooms,Tim Roth|Antonio Banderas|Jennifer Beals|Madon...,Allison Anders|Alexandre Rockwell|Robert Rodri...,98,Comedy,Miramax Films|A Band Apart,1995-12-25,293,6.4,5.723867e+06,6.153158e+06
1329,11,12.037933,Star Wars,Mark Hamill|Harrison Ford|Carrie Fisher|Peter ...,George Lucas,121,Adventure|Action|Science Fiction,Lucasfilm|Twentieth Century Fox Film Corporation,1977-03-20,4428,7.9,3.957559e+07,2.789712e+09
4955,12,3.440519,Finding Nemo,Albert Brooks|Ellen DeGeneres|Alexander Gould|...,Andrew Stanton|Lee Unkrich,100,Animation|Family,Walt Disney Pictures|Pixar Animation Studios|D...,2003-05-30,3692,7.4,1.114231e+08,1.024887e+09
4179,13,6.715966,Forrest Gump,Tom Hanks|Robin Wright|Gary Sinise|Mykelti Wil...,Robert Zemeckis,142,Comedy|Drama|Romance,Paramount Pictures,1994-07-06,4856,8.1,8.091114e+07,9.973333e+08
2411,14,3.555720,American Beauty,Kevin Spacey|Annette Bening|Thora Birch|Wes Be...,Sam Mendes,122,Drama,DreamWorks SKG|Jinks/Cohen Company,1999-09-15,1756,7.7,1.963579e+07,4.664111e+08


#### c. Cleaning Tideness Issues:

###### Define

6. Separate **release_date** into **release_day**, **release_month** and **release_year** (3 columns) and Dropping **release_date**.

###### Code

In [297]:
# extract year, month and day to new columns
tmdb_df_cleaned['release_day'] = tmdb_df_cleaned['release_date'].dt.day
tmdb_df_cleaned['release_month'] = tmdb_df_cleaned['release_date'].dt.month
tmdb_df_cleaned['release_year'] = tmdb_df_cleaned['release_date'].dt.year

# Finally drop timestamp column
tmdb_df_cleaned = tmdb_df_cleaned.drop('release_date', axis=1)

###### Test

In [298]:
tmdb_df_cleaned

,id,popularity,original_title,cast,director,runtime,genres,production_companies,vote_count,vote_average,budget_adj,revenue_adj,release_day,release_month,release_year
8088,5,1.234890,Four Rooms,Tim Roth|Antonio Banderas|Jennifer Beals|Madon...,Allison Anders|Alexandre Rockwell|Robert Rodri...,98,Comedy,Miramax Films|A Band Apart,293,6.4,5.723867e+06,6.153158e+06,25,12,1995
1329,11,12.037933,Star Wars,Mark Hamill|Harrison Ford|Carrie Fisher|Peter ...,George Lucas,121,Adventure|Action|Science Fiction,Lucasfilm|Twentieth Century Fox Film Corporation,4428,7.9,3.957559e+07,2.789712e+09,20,3,1977
4955,12,3.440519,Finding Nemo,Albert Brooks|Ellen DeGeneres|Alexander Gould|...,Andrew Stanton|Lee Unkrich,100,Animation|Family,Walt Disney Pictures|Pixar Animation Studios|D...,3692,7.4,1.114231e+08,1.024887e+09,30,5,2003
4179,13,6.715966,Forrest Gump,Tom Hanks|Robin Wright|Gary Sinise|Mykelti Wil...,Robert Zemeckis,142,Comedy|Drama|Romance,Paramount Pictures,4856,8.1,8.091114e+07,9.973333e+08,6,7,1994
2411,14,3.555720,American Beauty,Kevin Spacey|Annette Bening|Thora Birch|Wes Be...,Sam Mendes,122,Drama,DreamWorks SKG|Jinks/Cohen Company,1756,7.7,1.963579e+07,4.664111e+08,15,9,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,360387,0.393566,Blunt Force Trauma,Mickey Rourke|Freida Pinto|Ryan Kwanten|Maruia...,Ken Sanzel,92,Adventure|Action,ETA films,23,4.4,9.199996e+05,NaN,5,10,2015
330,362105,0.366030,R.L. Stine's Monsterville: The Cabinet of Souls,Dove Cameron|Katherine McNamara|Ryan McCartan|...,Peter DeLuise,85,Comedy|Horror,EveryWhere Studios,22,7.2,4.047998e+06,NaN,29,9,2015
515,395560,0.142759,Capsule,Edmund Kingsley|David Wayman|Nigel Barber|Lisa...,Andrew Martin,91,Drama|History|Thriller|Science Fiction,Ecaveo Capital Partners|Hermes Space Industries,11,5.3,1.195999e+06,NaN,23,12,2015
3826,414419,0.146477,Kill Bill: The Whole Bloody Affair,Uma Thurman|Lucy Liu|Vivica A. Fox|Daryl Hanna...,Quentin Tarantino,247,Crime|Action,Miramax Films|A Band Apart|Super Cool ManChu,28,8.1,2.908194e+07,NaN,28,3,2011


<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [299]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [300]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!